# Moving Targets: Basic Workflow

#### Imports

In [ ]:
import target
import plotting
import mast_tap

#### Target Parameters

Setting some parameters up front that will be used elsewhere.

In [ ]:
# 599 Luisa (A906 HF)
# obj_name = '599'  
# id_type = 'smallbody'
# times = {'start':'2010-01-01', 'stop':'2010-01-10', 'step':'1d'}

# Jupiter Barycenter
obj_name = '5'
id_type='majorbody'
times={'start': '2019-04-01', 'stop': '2019-04-10', 'step': '1d'} # WFC3
#times = {'start': '1997-07-10', 'stop': '1997-07-15', 'step': '12h'} # WFPC2

location = None # Geocentric default

#### Get the ephemerides as an astropy table

In [ ]:
eph = target.get_path(obj_name, id_type=id_type, times=times, location=location)

eph

#### Convert this to a space-time coordinate string polygon

See http://www.ivoa.net/documents/STC-S/20130917/WD-STC-S-1.0-20130917.html

In [ ]:
polygon = target.convert_path_to_polygon(eph)

polygon

#### Plotting the polygon

In [ ]:
plotting.polygon_bokeh(polygon, display=True)

#### TAP Query

In [ ]:
start_time = min(eph['datetime_jd']) - 2400000.5
end_time = max(eph['datetime_jd']) - 2400000.5

# If no start/end time supplied, any observation in that footprint would be matched
# results = mast_tap.run_tap_query(polygon, start_time=None, end_time=None, maxrec=100)

results = mast_tap.run_tap_query(polygon, start_time=start_time, end_time=end_time, maxrec=100)

# results

Filter results to only include the ones where the target was in the field of view of the observation. 

In [ ]:
filtered_results = mast_tap.clean_up_results(results, obj_name=obj_name, id_type=id_type, location=location)

filtered_results